In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv json-repair langchain-openai langchain_core

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_ollama import OllamaEmbeddings
import os
from langchain_experimental.llms.ollama_functions import OllamaFunctions
from neo4j import  Driver

from dotenv import load_dotenv

load_dotenv()

False

In [ ]:
NEO4J_URI="neo4j+s://ff9635e0.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="FlpkaG2M_jHx2yxE_HkdHFAn7oU2d-ldDxM2rrlvHC8"
AURA_INSTANCEID="ff9635e0"
AURA_INSTANCENAME="Instance01"

In [ ]:
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)

In [6]:
loader = TextLoader(file_path="/Users/akshit/Documents/Projects/Python-all/information-extraction/Guidelines/Graph/dummytext.txt")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=250, chunk_overlap=24)
documents = text_splitter.split_documents(documents=docs)

In [ ]:
!pip install -U langchain-ollama

In [7]:
llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")

llm_transformer = LLMGraphTransformer(llm=llm)

# graph_documents = llm_transformer.convert_to_graph_documents(documents)

/var/folders/bj/8gzt5tq51b1c9qlynhj982vc0000gn/T/ipykernel_45235/3456235371.py:1: LangChainDeprecationWarning: The class `OllamaFunctions` was deprecated in LangChain 0.0.64 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = OllamaFunctions(model="llama3.1", temperature=0, format="json")


In [8]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

OllamaEndpointNotFoundError: Ollama call failed with status code 404. Maybe your model is not found and you should pull the model with `ollama pull llama3.1`.

In [9]:
os.environ["OPENAI_API_KEY"] = <api_key>

In [ ]:
!pip install langchain_openai

In [10]:
''' use gpt '''
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

llm_transformer = LLMGraphTransformer(llm=llm)

In [11]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [12]:
print(graph_documents[0].nodes)
print(graph_documents[0].relationships)
print(graph_documents[0])

[Node(id='Node', type='Concept', properties={}), Node(id='Relationship', type='Concept', properties={}), Node(id='Node Classes', type='Concept', properties={}), Node(id='Relationship Classes', type='Concept', properties={}), Node(id='Customgraphdocument', type='Concept', properties={}), Node(id='Document Text', type='Concept', properties={})]
[Relationship(source=Node(id='Node Classes', type='Concept', properties={}), target=Node(id='Node', type='Concept', properties={}), type='MIRROR', properties={}), Relationship(source=Node(id='Relationship Classes', type='Concept', properties={}), target=Node(id='Relationship', type='Concept', properties={}), type='MIRROR', properties={}), Relationship(source=Node(id='Customgraphdocument', type='Concept', properties={}), target=Node(id='Document Text', type='Concept', properties={}), type='STORE', properties={}), Relationship(source=Node(id='Customgraphdocument', type='Concept', properties={}), target=Node(id='Node Classes', type='Concept', propert

In [13]:
print(len(graph_documents))
graph.add_graph_documents(graph_documents, baseEntityLabel=True)

5


NameError: name 'graph' is not defined

In [14]:
from langchain_community.graphs.graph_document import GraphDocument, Node, Relationship
from langchain_core.documents import Document

In [15]:
source_doc = Document(
    page_content='''http://nccn-guideline.org/nsclc/0,PROS-1,12,"Context:
INITIAL PROSTATE CANCER DIAGNOSIS{a,b,c}
Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX)

Next steps:
WORKUP
Perform physical exam
Perform digital rectal exam (DRE) to confirm clinical stage
Perform and/or collect prostate-specific antigen (PSA) and calculate PSA density
Obtain and review diagnostic prostate biopsies
Estimate life expectancy (Principles of Life Expectancy Estimation [PROS-A])
Inquire about known high-risk germline mutations and family history{d} Perform somatic and/or germline testing as appropriate{d}
Assess quality-of-life measures{e}","b: NCCN Guidelines for Prostate Cancer Early Detection.
d: Principles of Genetics and Molecular/Biomarker Analysis (PROS-C).
a: See NCCN Guidelines for Older Adult Oncology for tools to aid optimal assessment and management of disease in older adults.
e: Principles of Quality-of-Life and Shared Decision-Making (PROS-D).
c: Principles of Bone Health in Prostate Cancer (PROS-B).'''
)
movie = Node(id="INITIAL PROSTATE CANCER DIAGNOSIS", properties={"label": "cancer"})
actor1 = Node(id="Perform physical exam", properties={"label": "workup"})
actor2 = Node(
    id="Perform digital rectal exam (DRE) to confirm clinical stage", properties={"label": "workup"}
)
actor3 = Node(
    id="Inquire about known high-risk germline mutations and family history{d}", properties={"label": "workup"}
)
actor4 = Node(
    id="d: Principles of Genetics and Molecular/Biomarker Analysis (PROS-C).", properties={"label": "footnote"}
)
rel1 = Relationship(
    id=5, type="next steps", source=actor1, target=movie, properties={"label": "ActedIn"}
)
rel2 = Relationship(
    id=6, type="next steps", source=actor2, target=movie, properties={"label": "ActedIn"}
)
rel3 = Relationship(
    id=7, type="next steps", source=actor3, target=movie, properties={"label": "ActedIn"}
)
rel4 = Relationship(
    id=8,
    type="refer",
    source=actor3,
    target=actor4,
    properties={"label": "abc"},
)
# rel5 = Relationship(
#     id=9,
#     type="",
#     source=movie,
#     target=actor2,
#     properties={"label": "Strarring"},
# )
# rel6 = Relationship(
#     id=10,
#     type="Straring",
#     source=movie,
#     target=actor3,
#     properties={"label": "Strarring"},
# )
graph_doc = GraphDocument(
    nodes=[movie, actor1, actor2, actor3],
    relationships=[rel1, rel2, rel3, rel4],
    source=source_doc,
)

In [ ]:
graph.add_graph_documents([graph_doc])

In [ ]:
graph.refresh_schema()

In [ ]:
import json

# Load the JSON file
with open('example.json', 'r') as file:
    data = json.load(file)

# Initialize the result list
result_nodes = []

# Iterate through the @graph content
for node in data.get("@graph", []):
    # Check if the required keys are present
    if 'nccn:content' in node and node['nccn:content']:
      metadata = {
          "@id": node.get("@id"),
          "nccn:page-key": node.get("nccn:page-key"),
          "nccn:page-no": node.get("nccn:page-no"),
          "nccn:nodelink": node.get("nccn:nodelink")
      }
      result_nodes.append(Node(id=node['nccn:content'], properties=metadata))

# Output the result
print(result_nodes[0].properties['@id'])


http://nccn-guideline.org/nsclc/0


In [ ]:
node_lookup = {node.id: node for node in result_nodes}
target_node_lookup = {node.properties['@id']: node for node in result_nodes}

relationships = []
# Step 3: Build relationships based on 'nccn:next' entries
relationship_id = 1
for node in data.get("@graph", []):
    source_content = node.get("nccn:content")
    if source_content and 'nccn:next' in node:
        source_node = node_lookup.get(source_content)
        if source_node:
            for target_id in node['nccn:next']:
                print(target_id)
                target_node = target_node_lookup.get(target_id)
                if target_node:
                    # Direct relationship
                    relationships.append(Relationship(
                        id=relationship_id,
                        type="next step",
                        source=source_node,
                        target=target_node,
                        properties={}  # Add additional properties if needed
                    ))
                    relationship_id += 1
                else:
                    # Handle case where target node's nccn:content is missing
                    target_graph_node = next(
                        (g_node for g_node in data.get("@graph", []) if g_node.get("@id") == target_id),
                        None
                    )
                    if target_graph_node and 'nccn:contains' in target_graph_node:
                        for nested_id in target_graph_node['nccn:contains']:
                            nested_target_node = target_node_lookup.get(nested_id)
                            if nested_target_node:
                                # Create relationship for nested target nodes
                                relationships.append(Relationship(
                                    id=relationship_id,
                                    type="next step",
                                    source=source_node,
                                    target=nested_target_node,
                                    properties={}  # Add additional properties if needed
                                ))
                                relationship_id += 1

# Output the relationships


http://nccn-guideline.org/nsclc/1
http://nccn-guideline.org/nsclc/5
http://nccn-guideline.org/nsclc/5
http://nccn-guideline.org/nsclc/95
http://nccn-guideline.org/nsclc/97
http://nccn-guideline.org/nsclc/199
http://nccn-guideline.org/nsclc/24
http://nccn-guideline.org/nsclc/23
http://nccn-guideline.org/nsclc/27
Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX) Perform physical exam
Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX) Perform digital rectal exam (DRE) to confirm clinical stage
Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX) Perform and/or collect prostate-specific antigen (PSA) and calculate PSA density
Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX) Obtain and review diagnostic prostate biopsies
Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX) Estimate life expectancy (Principles of Life Expectancy Estimation [PROS-A])
Clinically localized prostate cancer (Any T, N0, M

In [ ]:
print(relationships[0])

source=Node(id='Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX)', properties={'@id': 'http://nccn-guideline.org/nsclc/0', 'nccn:page-key': 'PROS-1', 'nccn:page-no': 12, 'nccn:nodelink': 'http://localhost:8080/nscl.pdf?page=12&x=26.05&y=173.58&width=129.17&height=31.88'}) target=Node(id='Perform physical exam', properties={'@id': 'http://nccn-guideline.org/nsclc/8', 'nccn:page-key': 'PROS-1', 'nccn:page-no': 12, 'nccn:nodelink': 'http://localhost:8080/nscl.pdf?page=12&x=308.59&y=112.53&width=109.98&height=6.8'}) type='next step' properties={}


In [ ]:
GraphDocument(nodes=[Node(id="Amico'S Family", type='Family', properties={})], relationships=[],
              source=Document(metadata={'source': 'dummytext.txt', 'id': 'ed648f5744c61a703b28736f1a7cf0c0'},
              page_content='1. The Story of Amico’s Family: A Legacy of Love and Tradition'))

In [ ]:
a='''{
    "@graph": [
    {
        "@id": "http://nccn-guideline.org/nsclc/0",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=26.05&y=173.58&width=129.17&height=31.88",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=26.05,Y=173.58,W=129.17,H=31.88",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/0"
        ],
        "nccn:mscore": [
            "M0"
        ],
        "nccn:nscore": [
            "N0"
        ],
        "nccn:content": "Clinically localized prostate cancer (Any T, N0, M0 or Any T, NX, MX)",
        "nccn:next": [
            "http://nccn-guideline.org/nsclc/1"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/1",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=308.58&y=112.53&width=211.61&height=149.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=308.58,Y=112.53,W=211.61,H=149.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "",
        "nccn:prev": [
            "http://nccn-guideline.org/nsclc/0"
        ],
        "nccn:next": [
            "http://nccn-guideline.org/nsclc/2"
        ],
        "nccn:contains": [
            "http://nccn-guideline.org/nsclc/8",
            "http://nccn-guideline.org/nsclc/9",
            "http://nccn-guideline.org/nsclc/10",
            "http://nccn-guideline.org/nsclc/11",
            "http://nccn-guideline.org/nsclc/12",
            "http://nccn-guideline.org/nsclc/13",
            "http://nccn-guideline.org/nsclc/14"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/2",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=599.04&y=159.95&width=146.1&height=43.88",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=599.04,Y=159.95,W=146.1,H=43.88",
        "nccn:labels": [],
        "nccn:content": "See Initial Risk Stratification and Staging Workup for Clinically Localized Disease (PROS-2)",
        "nccn:prev": [
            "http://nccn-guideline.org/nsclc/1"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/3",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=23.55&y=303.08&width=116.76&height=19.88",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=23.55,Y=303.08,W=116.76,H=19.88",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/0"
        ],
        "nccn:mscore": [
            "M0"
        ],
        "nccn:nscore": [
            "N1"
        ],
        "nccn:content": "Regional prostate cancer (Any T, N1, M0)",
        "nccn:next": [
            "http://nccn-guideline.org/nsclc/5"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/4",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=23.55&y=382.44&width=134.07&height=19.88",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=23.55,Y=382.44,W=134.07,H=19.88",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/0"
        ],
        "nccn:mscore": [
            "M1"
        ],
        "nccn:content": "Metastatic prostate cancer (Any T, Any N, M1)",
        "nccn:next": [
            "http://nccn-guideline.org/nsclc/5"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/5",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=307.84&y=281.31&width=215.92&height=127.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=307.84,Y=281.31,W=215.92,H=127.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "",
        "nccn:prev": [
            "http://nccn-guideline.org/nsclc/3",
            "http://nccn-guideline.org/nsclc/4"
        ],
        "nccn:next": [
            "http://nccn-guideline.org/nsclc/7",
            "http://nccn-guideline.org/nsclc/6"
        ],
        "nccn:contains": [
            "http://nccn-guideline.org/nsclc/15",
            "http://nccn-guideline.org/nsclc/16",
            "http://nccn-guideline.org/nsclc/17",
            "http://nccn-guideline.org/nsclc/18",
            "http://nccn-guideline.org/nsclc/19",
            "http://nccn-guideline.org/nsclc/20",
            "http://nccn-guideline.org/nsclc/21"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/6",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=597.71&y=300.52&width=119.21&height=19.88",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=597.71,Y=300.52,W=119.21,H=19.88",
        "nccn:labels": [],
        "nccn:content": "See Regional Prostate Cancer (PROS-8)",
        "nccn:prev": [
            "http://nccn-guideline.org/nsclc/5"
        ],
        "nccn:next": [
            "http://nccn-guideline.org/nsclc/95",
            "http://nccn-guideline.org/nsclc/97"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/7",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=597.71&y=377.57&width=156.49&height=31.88",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=597.71,Y=377.57,W=156.49,H=31.88",
        "nccn:labels": [],
        "nccn:mscore": [
            "M1"
        ],
        "nccn:content": "Systemic Therapy for M1 Castration-Sensitive Prostate Cancer (CSPC) (PROS-13)",
        "nccn:prev": [
            "http://nccn-guideline.org/nsclc/5"
        ],
        "nccn:next": [
            "http://nccn-guideline.org/nsclc/199"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/8",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=308.59&y=112.53&width=109.98&height=6.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=308.59,Y=112.53,W=109.98,H=6.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Perform physical exam",
        "nccn:parent": "http://nccn-guideline.org/nsclc/1"
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/9",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=308.59&y=123.53&width=208.68&height=17.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=308.59,Y=123.53,W=208.68,H=17.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Perform digital rectal exam (DRE) to confirm clinical stage",
        "nccn:parent": "http://nccn-guideline.org/nsclc/1"
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/10",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=308.59&y=145.53&width=190.03&height=17.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=308.59,Y=145.53,W=190.03,H=17.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Perform and/or collect prostate-specific antigen (PSA) and calculate PSA density",
        "nccn:parent": "http://nccn-guideline.org/nsclc/1"
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/11",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=308.59&y=167.54&width=180.71&height=17.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=308.59,Y=167.54,W=180.71,H=17.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Obtain and review diagnostic prostate biopsies",
        "nccn:parent": "http://nccn-guideline.org/nsclc/1"
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/12",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=308.59&y=189.54&width=201.31&height=17.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=308.59,Y=189.54,W=201.31,H=17.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Estimate life expectancy (Principles of Life Expectancy Estimation [PROS-A])",
        "nccn:parent": "http://nccn-guideline.org/nsclc/1"
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/13",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=308.58&y=211.54&width=211.61&height=39.79",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=308.58,Y=211.54,W=211.61,H=39.79",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Inquire about known high-risk germline mutations and family history{d} Perform somatic and/or germline testing as appropriate{d}",
        "nccn:parent": "http://nccn-guideline.org/nsclc/1",
        "nccn:reference": [
            "http://nccn-guideline.org/nsclc/footnote/d"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/14",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=308.59&y=255.53&width=152.97&height=6.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=308.59,Y=255.53,W=152.97,H=6.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Assess quality-of-life measures{e}",
        "nccn:parent": "http://nccn-guideline.org/nsclc/1",
        "nccn:reference": [
            "http://nccn-guideline.org/nsclc/footnote/e"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/15",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=307.85&y=281.31&width=112.62&height=6.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=307.85,Y=281.31,W=112.62,H=6.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Perform physical exam",
        "nccn:parent": "http://nccn-guideline.org/nsclc/5"
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/16",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=307.85&y=292.31&width=150.06&height=6.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=307.85,Y=292.31,W=150.06,H=6.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Perform imaging for staging{f,g,h}",
        "nccn:parent": "http://nccn-guideline.org/nsclc/5",
        "nccn:reference": [
            "http://nccn-guideline.org/nsclc/footnote/f",
            "http://nccn-guideline.org/nsclc/footnote/g",
            "http://nccn-guideline.org/nsclc/footnote/h"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/17",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=307.85&y=303.31&width=176.48&height=6.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=307.85,Y=303.31,W=176.48,H=6.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Perform DRE to confirm clinical stage",
        "nccn:parent": "http://nccn-guideline.org/nsclc/5"
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/18",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=307.85&y=314.31&width=215.91&height=17.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=307.85,Y=314.31,W=215.91,H=17.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Perform and/or collect PSA and calculate PSA doubling time (PSADT)",
        "nccn:parent": "http://nccn-guideline.org/nsclc/5"
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/19",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=307.85&y=336.31&width=201.31&height=17.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=307.85,Y=336.31,W=201.31,H=17.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Estimate life expectancy (Principles of Life Expectancy Estimation [PROS-A])",
        "nccn:parent": "http://nccn-guideline.org/nsclc/5"
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/20",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=307.84&y=358.31&width=211.61&height=39.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=307.84,Y=358.31,W=211.61,H=39.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Inquire about known high-risk germline mutations and family history{d} Perform somatic and/or germline testing as appropriate{d}",
        "nccn:parent": "http://nccn-guideline.org/nsclc/5",
        "nccn:reference": [
            "http://nccn-guideline.org/nsclc/footnote/d"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/21",
        "nccn:page-key": "PROS-1",
        "nccn:page-no": 12,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=12&x=307.85&y=402.31&width=152.97&height=6.8",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=12",
        "nccn:bounds": "X=307.85,Y=402.31,W=152.97,H=6.8",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/1"
        ],
        "nccn:content": "Assess quality-of-life measures{e}",
        "nccn:parent": "http://nccn-guideline.org/nsclc/5",
        "nccn:reference": [
            "http://nccn-guideline.org/nsclc/footnote/e"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/22",
        "nccn:page-key": "PROS-3",
        "nccn:page-no": 15,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=15&x=23.55&y=228.38&width=32.94&height=7.88",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=15",
        "nccn:bounds": "X=23.55,Y=228.38,W=32.94,H=7.88",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/4",
            "http://nccn-guideline.org/nsclc/labels/5"
        ],
        "nccn:content": "≥10 y{o}",
        "nccn:next": [
            "http://nccn-guideline.org/nsclc/24"
        ],
        "nccn:reference": [
            "http://nccn-guideline.org/nsclc/footnote/o"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/23",
        "nccn:page-key": "PROS-3",
        "nccn:page-no": 15,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=15&x=564.57&y=204.37&width=151.0&height=55.88",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=15",
        "nccn:bounds": "X=564.57,Y=204.37,W=151.0,H=55.88",
        "nccn:labels": [],
        "nccn:content": "Progressive disease{s} See Initial Risk Stratification and Staging Workup for Clinically Localized Disease (PROS-2)",
        "nccn:prev": [
            "http://nccn-guideline.org/nsclc/24"
        ],
        "nccn:reference": [
            "http://nccn-guideline.org/nsclc/footnote/s"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/24",
        "nccn:page-key": "PROS-3",
        "nccn:page-no": 15,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=15&x=152.58&y=216.37&width=174.45&height=31.88",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=15",
        "nccn:bounds": "X=152.58,Y=216.37,W=174.45,H=31.88",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/4",
            "http://nccn-guideline.org/nsclc/labels/6"
        ],
        "nccn:content": "Active surveillance{p,q} See Active Surveillance Program (PROS-F 2 of 5)",
        "nccn:prev": [
            "http://nccn-guideline.org/nsclc/22"
        ],
        "nccn:next": [
            "http://nccn-guideline.org/nsclc/23"
        ],
        "nccn:reference": [
            "http://nccn-guideline.org/nsclc/footnote/p",
            "http://nccn-guideline.org/nsclc/footnote/q"
        ]
    },
    {
        "@id": "http://nccn-guideline.org/nsclc/25",
        "nccn:page-key": "PROS-3",
        "nccn:page-no": 15,
        "nccn:nodelink": "http://localhost:8080/nscl.pdf?page=15&x=152.58&y=372.14&width=68.13&height=7.88",
        "nccn:pagelink": "http://localhost:8080/nscl.pdf#page=15",
        "nccn:bounds": "X=152.58,Y=372.14,W=68.13,H=7.88",
        "nccn:labels": [
            "http://nccn-guideline.org/nsclc/labels/4",
            "http://nccn-guideline.org/nsclc/labels/6"
        ],
        "nccn:content": "Observationr",
        "nccn:prev": [
            "http://nccn-guideline.org/nsclc/26"
        ],
        "nccn:next": [
            "http://nccn-guideline.org/nsclc/27"
        ],
        "nccn:reference": [
            "http://nccn-guideline.org/nsclc/footnote/j"
        ]
    }]
}'''

In [ ]:
source_doc = Document(
    page_content=a)
graph_doc = GraphDocument(
    nodes=result_nodes,
    relationships=relationships,
    source=source_doc,
)

In [ ]:
graph.add_graph_documents([graph_doc])

In [ ]:
graph.refresh_schema()

In [ ]:
print(len(result_nodes))
print(len(relationships))

24
23
